In [1]:
import os
os.sys.path.append("../")
from scripts.geospatial_analysis import *

In [2]:
# Create a Spark Session
spark = (
    SparkSession.builder.appName("geospatial analysis")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.execturo.memory", "2g")
    .getOrCreate()
)

24/09/10 15:34:00 WARN Utils: Your hostname, qinsitaodeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.13.18.159 instead (on interface en0)
24/09/10 15:34:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/10 15:34:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/10 15:34:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/10 15:34:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/10 15:34:02 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
consumer_info = spark.read.parquet('../data/curated/consumer_info.parquet')
fraudulent_consumer_rate = spark.read.parquet('../data/curated/consumer_fraud_prob.parquet')

In [4]:
fraudulent_consumer_with_info = consumer_info.join(fraudulent_consumer_rate, on="consumer_id", how="inner")

In [5]:
# Find info for fraudulent consumer
fraudulent_consumer_with_info = consumer_info.join(fraudulent_consumer_rate, on="consumer_id", how="inner")

In [6]:
# Number of consumer in each state
consumer_info.groupby("state").agg(F.count("consumer_id"))

state,count(consumer_id)
NT,7764
ACT,4664
SA,54973
TAS,18878
WA,79146
QLD,72861
VIC,117525
NSW,144188


In [7]:
# Number of fraudulent consumer in each state
fraudulent_consumer_with_info.groupby("state").agg(F.count("consumer_id"))

24/09/10 15:34:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


state,count(consumer_id)
NT,480
ACT,313
SA,3926
TAS,1332
WA,5538
QLD,5109
VIC,8129
NSW,10037


24/09/10 16:48:19 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 804446 ms exceeds timeout 120000 ms
24/09/10 16:48:19 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/10 16:48:25 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [8]:
fraudulent_consumer_with_info.count()
#consumer_info.count()
#fraudulent_consumer_with_info.select("consumer_id").distinct().count() # Number of unique fraudulent consumer

34864

In [9]:
# Check the number of NULL values
null_counts = fraudulent_consumer_with_info.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in fraudulent_consumer_with_info.columns])
null_counts.limit(1)
# null_counts = consumer_info.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in fraudulent_consumer_with_info.columns])
# null_counts.limit(1)


consumer_id,name,gender,state,postcode,order_datetime,fraud_probability
0,0,0,0,0,0,0


In [10]:
# Number of consumer in each postcode
consumer_info.groupBy(["postcode"]).agg(F.count("consumer_id").alias("total_consumer"))

postcode,total_consumer
1238,162
6466,176
2142,150
4101,166
5156,173
2122,170
3749,148
2866,137
3918,154
6336,157


# External dataset

ABS Digital Boundary files

`States and Territories - 2021 - Shapefile`

`Australian postcodes - Shapefile`

https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files#downloads-for-gda2020-digital-boundary-files


In [11]:
# Transform the full state name to the abbriviation name
state_abbr = {
    "New South Wales": "NSW",
    "Victoria": "VIC",
    "Queensland": "QLD",
    "South Australia": "SA",
    "Western Australia": "WA",
    "Tasmania": "TAS",
    "Northern Territory": "NT",
    "Australian Capital Territory": "ACT"
}

In [12]:
# Load the shapefile into a GeoDataFrame
postcode_shapefile_path = '../data/raw/POA_2021_AUST_GDA2020_SHP/POA_2021_AUST_GDA2020.shp'
gdf_postcode = gpd.read_file(postcode_shapefile_path)

state_shapefile_path = '../data/raw/STE_2021_AUST_SHP_GDA2020/STE_2021_AUST_GDA2020.shp'
gdf_state = gpd.read_file(state_shapefile_path)
gdf_state['STE_NAME21'] = gdf_state['STE_NAME21'].replace(state_abbr)

In [13]:
fraudulent_consumer_with_info

consumer_id,name,gender,state,postcode,order_datetime,fraud_probability
1195503,Yolanda Williams,Female,WA,6935,2022-02-20,9.805431136520959
179208,Mary Smith,Female,NSW,2782,2021-09-25,10.069850934775245
179208,Mary Smith,Female,NSW,2782,2021-08-30,9.599513915425788
1194530,Jill Jones MD,Female,NT,862,2021-11-03,8.300636455314633
154128,Lindsay Jimenez,Female,NSW,2780,2021-10-09,9.633302411090419
712975,Rebecca Blanchard,Female,WA,6355,2022-01-11,27.496186536467164
712975,Rebecca Blanchard,Female,WA,6355,2021-10-04,10.868364868449886
712975,Rebecca Blanchard,Female,WA,6355,2022-02-08,9.02022421158597
407340,Karen Chapman,Female,NSW,2033,2021-12-12,10.459280127078758
650435,Stephanie Reyes,Female,NSW,2482,2021-12-13,10.58055311139687


In [14]:
# Number of (fraudulent) consumer in each postcode or state
consumer_group_by_postcode = consumer_info.groupBy(["postcode"]).agg(F.count("consumer_id").alias("total_consumer"))
consumer_group_by_state = consumer_info.groupBy(["state"]).agg(F.count("consumer_id").alias("total_consumer"))

fraudulent_consumer_group_by_postcode = fraudulent_consumer_with_info.groupBy(["postcode"]).agg(F.count("consumer_id").alias("total_consumer"),
                                                                                                 F.avg("fraud_probability").alias("average_fraud_prob"),
                                                                                                 F.max("fraud_probability").alias("max_fraud_probability"))
fraudulent_consumer_group_by_state = fraudulent_consumer_with_info.groupBy(["state"]).agg(F.count("consumer_id").alias("total_consumer"),
                                                                                                 F.avg("fraud_probability").alias("average_fraud_prob"),
                                                                                                 F.max("fraud_probability").alias("max_fraud_probability"))

In [15]:
# Group the consumer and fraudulent consumer by different features
consumer_group_by_postcode = consumer_group_by_postcode.toPandas()
consumer_group_by_state = consumer_group_by_state.toPandas()

fraudulent_consumer_group_by_postcode = fraudulent_consumer_group_by_postcode.toPandas()
fraudulent_consumer_group_by_state = fraudulent_consumer_group_by_state.toPandas()

consumer_group_by_postcode['postcode'] = consumer_group_by_postcode['postcode'].astype(str)
consumer_group_by_state['state'] = consumer_group_by_state['state'].astype(str)

fraudulent_consumer_group_by_postcode['postcode'] = fraudulent_consumer_group_by_postcode['postcode'].astype(str)
fraudulent_consumer_group_by_state['state'] = fraudulent_consumer_group_by_state['state'].astype(str)

gdf_postcode['POA_CODE21'] = gdf_postcode['POA_CODE21'].astype(str)

In [16]:
# Merge consumer information with geographic data from the shapefile
total_consumer_group_by_postcode_merged_gdf = gdf_postcode.merge(consumer_group_by_postcode, left_on='POA_CODE21', right_on='postcode')
total_consumer_group_by_postcode__geojson = total_consumer_group_by_postcode_merged_gdf.to_json()

fraudulent_consumer_group_by_postcode_merged_gdf = gdf_postcode.merge(fraudulent_consumer_group_by_postcode, left_on='POA_CODE21', right_on='postcode')
fraudulent_consumer_group_by_postcode_geojson = fraudulent_consumer_group_by_postcode_merged_gdf.to_json()

In [17]:
# Merge consumer information with geographic data from the shapefile
total_consumer_group_by_state_merged_gdf = gdf_state.merge(consumer_group_by_state, left_on='STE_NAME21', right_on='state')
total_consumer_group_by_state__geojson = total_consumer_group_by_state_merged_gdf.to_json()

fraudulent_consumer_group_by_state_merged_gdf = gdf_state.merge(fraudulent_consumer_group_by_state, left_on='STE_NAME21', right_on='state')
fraudulent_consumer_group_by_state_geojson = fraudulent_consumer_group_by_state_merged_gdf.to_json()

In [ ]:
# Distribution of number of customer group by postcode
# Computer might be lagging, better to clear output of this cell
create_consumer_map(total_consumer_group_by_postcode__geojson,consumer_group_by_postcode, "postcode", "total_consumer", "POA_CODE21")

In [ ]:
# Distribution of average fraud probability group by postcode
create_consumer_map(fraudulent_consumer_group_by_postcode_geojson,fraudulent_consumer_group_by_postcode, "postcode", "average_fraud_prob", "POA_CODE21")

In [ ]:
# Distribution of maximum fraud probability group by postcode
create_consumer_map(fraudulent_consumer_group_by_postcode_geojson,fraudulent_consumer_group_by_postcode, "postcode", "max_fraud_probability", "POA_CODE21")

In [ ]:
# Group by state instead
create_consumer_map(total_consumer_group_by_state__geojson,consumer_group_by_state, "state", "total_consumer","STE_NAME21")

In [ ]:
create_consumer_map(fraudulent_consumer_group_by_state_geojson,fraudulent_consumer_group_by_state, "state", "average_fraud_prob","STE_NAME21")

In [ ]:
create_consumer_map(fraudulent_consumer_group_by_state_geojson,fraudulent_consumer_group_by_state, "state", "max_fraud_probability","STE_NAME21")